In [ ]:
from keras.models import Sequential
import keras
import numpy as np
import matplotlib.pyplot as plt 

import random
from keras.layers import Conv2D,MaxPooling2D,Flatten,Softmax,Activation,Dense,BatchNormalization

from keras.callbacks import TensorBoard
from keras.datasets import mnist,cifar10
from keras.models import Model
from keras.layers import Input, Flatten, Dense, Dropout, Lambda
from keras.optimizers import *
from keras import backend as K

num_classes = 5
epochs = 10

In [ ]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"
os.environ['CUDA_VISIBLE_DEVICES']='0'
import tensorflow as tf
print(tf.test.is_gpu_available())

In [ ]:
l_0=list(os.listdir('dataset0/0'))
l_1=list(os.listdir('dataset0/1'))
l_2=list(os.listdir('dataset0/2'))
l_3=list(os.listdir('dataset0/3'))
l_4=list(os.listdir('dataset0/4'))

In [ ]:
print(len(l_0))
print(len(l_1))
print(len(l_2))
print(len(l_3))
print(len(l_4))

In [ ]:
image_types = (".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff")
def list_images(basePath, contains=None):
    # 返回有效的图片路径数据集
    return list_files(basePath, validExts=image_types, contains=contains)
def list_files(basePath, validExts=None, contains=None):
    # 遍历图片数据目录，生成每张图片的路径
    for (rootDir, dirNames, filenames) in os.walk(basePath):
        # 循环遍历当前目录中的文件名
        for filename in filenames:
            # if the contains string is not none and the filename does not contain
            # the supplied string, then ignore the file
            if contains is not None and filename.find(contains) == -1:
                continue
             # 通过确定.的位置，从而确定当前文件的文件扩展名
            ext = filename[filename.rfind("."):].lower()
            
            # 检查文件是否为图像，是否应进行处理
            if validExts is None or ext.endswith(validExts):
                # 构造图像路径
                imagePath = os.path.join(rootDir, filename)
                yield imagePath

In [ ]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
import random
import pickle
import cv2
import os
from keras.utils.np_utils import to_categorical
# 读取数据和标签
print("------开始读取数据------")
data = []
labels = []

imagePaths = sorted(list(list_images('dataset0')))
random.seed(0)
random.shuffle(imagePaths)
for imagePath in imagePaths:
    image = cv2.imread(imagePath)

    data.append(image)
    
    label = imagePath.split(os.path.sep)[-2]
    labels.append(label)
    if len(data)==15000:
        break
print(len(data))

In [ ]:
labels

In [ ]:
(trainX, testX, trainY, testY) = train_test_split(data,labels, test_size=0.1, random_state=72)

In [ ]:

trainX=np.array(trainX)
testX=np.array(testX)
trainX = trainX.astype('float32')
testX = testX.astype('float32')
trainX /= 255
testX /= 255
trainY=np.array(trainY)
print(type(trainY))
testY=np.array(testY)

In [ ]:
trainY[0:10]

In [ ]:

trainY = [ int(x) for x in trainY ]
testY = [int(x) for x in testY ]
trainY=np.array(trainY)
testY=np.array(testY)
print(trainX.shape) 
print(trainY.shape)
print(testX.shape)
print(testY.shape)

In [ ]:
plt.imshow(trainX[100])

In [ ]:
trainY[100]

In [ ]:
random.randrange(1, 5)

In [ ]:
def euclidean_distance(vects):
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))


def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)


def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    margin = 1
    sqaure_pred = K.square(y_pred)
    margin_square = K.square(K.maximum(margin - y_pred, 0))
    return K.mean(y_true * sqaure_pred + (1 - y_true) * margin_square)



from keras.layers import *
def identity_block(X, f, filters, stage, block):
    conv_name_base = "res" + str(stage) + block + "_branch"
    bn_name_base   = "bn"  + str(stage) + block + "_branch"
    
    F1, F2, F3 = filters
    
    X_shortcut = X
    
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding="valid", 
               name=conv_name_base+"2a", kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + "2a")(X)
    X = Activation("relu")(X)
    
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding="same",
               name=conv_name_base+"2b", kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base+"2b")(X)
    X = Activation("relu")(X)
    # Third component of main path (≈2 lines)


    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding="valid",
               name=conv_name_base+"2c", kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base+"2c")(X)
    
    X = Add()([X, X_shortcut])
    X = Activation("relu")(X)
    ### END CODE HERE ###
    
    return X

def convolutional_block(X, f, filters, stage, block, s = 2):
    
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    ##### MAIN PATH #####
    # First component of main path 
    X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a', padding='valid', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    ### START CODE HERE ###

    # Second component of main path (≈3 lines)
    X = Conv2D(F2, (f, f), strides = (1, 1), name = conv_name_base + '2b',padding='same', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = Conv2D(F3, (1, 1), strides = (1, 1), name = conv_name_base + '2c',padding='valid', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    ##### SHORTCUT PATH #### (≈2 lines)
    X_shortcut = Conv2D(F3, (1, 1), strides = (s, s), name = conv_name_base + '1',padding='valid', kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = layers.add([X, X_shortcut])
    X = Activation('relu')(X)
    
    ### END CODE HERE ###
    
    return X
 

def MyNet(input_shape = (64, 64, 3), classes = 5):
    """
    Implementation of the popular ResNet50 the following architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    
    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(filters=64, kernel_size=(7, 7), strides=(2, 2), name="conv",
               kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name="bn_conv1")(X)
    X = Activation("relu")(X)
    X = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f=3, filters=[64, 64,256], stage=2, block="a", s=2)
    X = identity_block(X, f=3, filters=[64, 64, 256], stage=2, block="b")
    X = identity_block(X, f=3, filters=[64, 64, 256], stage=2, block="c")
    ### START CODE HERE ###

    # Stage 3 (≈4 lines)
    # The convolutional block uses three set of filters of size [128,128,512], "f" is 3, "s" is 2 and the block is "a".
    # The 3 identity blocks use three set of filters of size [128,128,512], "f" is 3 and the blocks are "b", "c" and "d".
    X = convolutional_block(X, f=3, filters=[128, 128, 512], stage=3, block="a", s=2)
    X = identity_block(X, f=3, filters=[128, 128, 512], stage=3, block="b")
    X = identity_block(X, f=3, filters=[128, 128, 512], stage=3, block="c")
    X = identity_block(X, f=3, filters=[128, 128, 512], stage=3, block="d")
    
    # Stage 4 (≈6 lines)
    # The convolutional block uses three set of filters of size [256, 256, 1024], "f" is 3, "s" is 2 and the block is "a".
    # The 5 identity blocks use three set of filters of size [256, 256, 1024], "f" is 3 and the blocks are "b", "c", "d", "e" and "f".
    X = convolutional_block(X, f=3, filters=[256, 256, 1024], stage=4, block="a", s=1)
    X = identity_block(X, f=3, filters=[256, 256, 1024], stage=4, block="b")
    X = identity_block(X, f=3, filters=[256, 256, 1024], stage=4, block="c")
    X = identity_block(X, f=3, filters=[256, 256, 1024], stage=4, block="d")
    

  
    
    X = AveragePooling2D(pool_size=(1, 1), padding="same")(X)
    X = Flatten()(X)
    X = Dense(1000)(X)
    X = Dense(128)(X)

    model = Model(inputs=X_input, outputs=X, name="ResNet50")
    model.summary()
    return model
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    pred = y_pred.ravel() < 0.5
    return np.mean(pred == y_true)


def accuracy(y_true, y_pred): # Tensor上的操作
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    return K.mean(K.equal(y_true, K.cast(y_pred < 0.5, y_true.dtype)))

def plot_train_history(history, train_metrics, val_metrics):
    plt.plot(history.history.get(train_metrics))
    plt.plot(history.history.get(val_metrics))
    plt.ylabel(train_metrics)
    plt.xlabel('Epochs')
    plt.legend(['train', 'validation'])
 

In [ ]:
lis=[0,0,0,0,0]
for i in range(len(trainY)):
   lis[int(trainY[i])]+=1
print(lis)

In [ ]:
plt.imshow(trainX[0])

In [ ]:
trainY

In [ ]:
# create training+test positive and negative pairs
digit_indices = [np.where(trainY== i)[0] for i in range(5)]
print(len(digit_indices))
print(type(digit_indices))
print(digit_indices[0].shape)
print(digit_indices[1].shape)
print(digit_indices[2].shape)
print(digit_indices[3].shape)
print(digit_indices[4].shape)

In [ ]:
def create_pairs(x, digit_indices):
    '''Positive and negative pair creation.
    Alternates between positive and negative pairs.
    '''
    pairs = []
    labels = []
#     n = min([len(digit_indices[d]) for d in range(num_classes)]) - 1
    for d in range(num_classes):
        n= len(digit_indices[d])-1
        for i in range(n):
            z1, z2 = digit_indices[d][i], digit_indices[d][i + 1]
            pairs += [[x[z1], x[z2]]]
            inc = random.randrange(1, num_classes)
            dn = (d + inc) % num_classes
            if(i>len(digit_indices[dn])-1):
                labels += [1]
                continue
            z1, z2 = digit_indices[d][i], digit_indices[dn][i]
            pairs += [[x[z1], x[z2]]]
            labels += [1, 0]
    return np.array(pairs), np.array(labels)

In [ ]:
tr_pairs, tr_y = create_pairs(trainX, digit_indices)
print(tr_pairs.shape)
print(type(tr_pairs))
print(len(tr_y))

In [ ]:
digit_indices = [np.where( testY == i)[0] for i in range(5)]
te_pairs, te_y = create_pairs(testX, digit_indices)
print(te_pairs.shape)

In [ ]:
input_shape = trainX.shape[1:]
print(input_shape)

In [ ]:
import keras.backend as K
from keras.callbacks import LearningRateScheduler
from keras.callbacks import EarlyStopping

# def scheduler(epoch):
#     # 每隔10个epoch，学习率减小为原来的1/10
#     if epoch % 10 == 0 and epoch != 0:
#         lr = K.get_value(model.optimizer.lr)
#         K.set_value(model.optimizer.lr, lr * 0.1)
#         print("lr changed to {}".format(lr * 0.1))
#     return K.get_value(model.optimizer.lr)
def scheduler(epoch):
 
    if epoch == 10 and epoch != 0:
        lr = K.get_value(model.optimizer.lr)
        K.set_value(model.optimizer.lr, lr * 0.1)
        print("lr changed to {}".format(lr * 0.1))
    if epoch == 20 and epoch != 0:
        lr = K.get_value(model.optimizer.lr)
        K.set_value(model.optimizer.lr, lr * 10)
        print("lr changed to {}".format(lr * 10))
    if epoch == 30 and epoch != 0:
        lr = K.get_value(model.optimizer.lr)
        K.set_value(model.optimizer.lr, lr * 0.1)
        print("lr changed to {}".format(lr * 0.1))
    if epoch == 40 and epoch != 0:
        lr = K.get_value(model.optimizer.lr)
        K.set_value(model.optimizer.lr, lr * 10)
        print("lr changed to {}".format(lr * 10))
    return K.get_value(model.optimizer.lr)
reduce_lr = LearningRateScheduler(scheduler)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)
for i in range(10):
    print("++++++++++++",i,"+++++++++++++++++")
    base_network = MyNet(input_shape)

    input_a = Input(shape=input_shape)
    input_b = Input(shape=input_shape)

    # because we re-use the same instance `base_network`,
    # the weights of the network
    # will be shared across the two branches
    processed_a = base_network(input_a)
    processed_b = base_network(input_b)

    distance = Lambda(euclidean_distance,
                      output_shape=eucl_dist_output_shape)([processed_a, processed_b])

    model = Model([input_a, input_b], distance)
 
    opt =RMSprop(lr=0.001)
    model.compile(loss=contrastive_loss, optimizer=opt, metrics=[accuracy])
    history=model.fit([tr_pairs[:, 0], tr_pairs[:, 1]], tr_y,
              batch_size=32,
              epochs=40,verbose=1,
              callbacks=[reduce_lr],
    #                   ,early_stopping
              validation_data=([te_pairs[:, 0], te_pairs[:, 1]], te_y))

In [ ]:
# compute final accuracy on training and test sets
yr_pred = model.predict([tr_pairs[:, 0], tr_pairs[:, 1]])
tr_acc = compute_accuracy(tr_y, yr_pred)
ye_pred = model.predict([te_pairs[:, 0], te_pairs[:, 1]])
te_acc = compute_accuracy(te_y, ye_pred)

print('* Accuracy on training set: %0.2f%%' % (100 * tr_acc))
print('* Accuracy on test set: %0.2f%%' % (100 * te_acc))


In [ ]:
plt.figure(figsize=(20,8))

plt.subplot(1, 2, 1)
plt.plot(history.history.get("accuracy")[0:],"green")
plt.plot(history.history.get("val_accuracy")[0:],color='#054E9F')
plt.ylabel("acc")
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
plt.grid(alpha=1,linestyle=':')

plt.subplot(1, 2, 2)
plt.plot(history.history.get("loss")[2:],"green")
plt.plot(history.history.get("val_loss")[2:],color='#054E9F')
plt.ylabel("loss")
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
plt.grid(alpha=1,linestyle=':')

In [ ]:
plt.figure(figsize=(8, 4))
plt.subplot(1, 2, 1)

plt.plot(history.history.get("loss")[0:])
plt.plot(history.history.get("val_loss")[0:])
plt.ylabel("loss")
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
 
plt.subplot(1, 2, 2)

plt.plot(history.history.get("accuracy")[0:])
plt.plot(history.history.get("val_accuracy")[0:])
plt.ylabel("acc")
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
 
plt.show()


In [ ]:
from sklearn.metrics import roc_curve
 
fpr, tpr, thresholds_keras = roc_curve(tr_y[:,0], y_pred[:,0])
auc_val = auc(fpr, tpr)
print("AUC : ",auc_val)
plt.figure()
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr, tpr, label='AUC = {:.3f})'.format(auc_val))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()

In [ ]:
tr_y

In [ ]:
y_pred

In [ ]:
n=0
for i in range(len(yr_pred)):
    if(tr_y[i]==1 and yr_pred[i]==1 or tr_y[i]==0 and yr_pred[i]==0 ):
        n+=1
    print(n)

In [ ]:
n

In [ ]:
len(tr_y)

In [ ]:
41677/42779

In [ ]:
for i in range(len(yr_pred)):
    if(yr_pred[i]>0.5):
        yr_pred[i]=0
    else:
        yr_pred[i]=1

In [ ]:
yr_pred

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

fpr, tpr, thresholds_keras = roc_curve(tr_y, yr_pred)
auc_val = auc(fpr, tpr)
print("AUC : ",auc_val)
plt.figure()
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr, tpr, label='AUC = {:.3f})'.format(auc_val))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()